In [118]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('spark') \
    .getOrCreate()

In [13]:
## 1. OTs GS

ot_gs = spark.read.json('data/gwas_gold_standards.191108.jsonl')

In [17]:
print(ot_gs.first())
#ot_gs.printSchema()

Row(association_info=Row(ancestry=['EUR'], doi=None, gwas_catalog_id='GCST000324', neg_log_pval=23.699, otg_id='GCST000324_3', pubmed_id='19185284', url=None), gold_standard_info=Row(evidence=[Row(class='expert curated', confidence='High', curated_by='Eric Fauman', description='BCO1 (previously referred to as BCMO1) encodes beta-carotene oxygenase 1 which uses a molecule of oxygen to produce two molecules of retinol from beta-carotene.  Enzyme deficiency results in accumulation of beta-carotene.', pubmed_id='11401432', source=None)], gene_id='ENSG00000135697', highest_confidence='High'), metadata=Row(comments=None, date_added='2019-05-17', reviewed_by='Ed Mountjoy', set_label='ProGeM', submitted_by='Eric Fauman', tags=['metabolite', 'mQTL']), sentinel_variant=Row(alleles=Row(alternative='G', reference='T'), locus_GRCh37=Row(chromosome='16', position=81264597), locus_GRCh38=Row(chromosome='16', position=81230992), rsid='rs6564851'), trait_info=Row(ontology=['HMDB0000561'], reported_trai

In [36]:
# Provenances of the assoc
print(ot_gs.select('metadata.tags').distinct().show(truncate=False))

#print(ot_gs.filter(F.col('metadata.tags').isNull()).first())

+---------------------------------+
|tags                             |
+---------------------------------+
|[drug, ChEMBL, ChEMBL_III]       |
|null                             |
|[metabolite, mQTL]               |
|[drug, ChEMBL, ChEMBL_II]        |
|[T2D]                            |
|[Fauman, Biological plausability]|
|[drug, ChEMBL, ChEMBL_IV]        |
+---------------------------------+

None


In [37]:
ot_gs = (ot_gs
 .filter(F.col('gold_standard_info.highest_confidence').isin(['High', 'Medium']))
 # Extract gene/trait pairs
 .withColumn('trait_id', F.explode('trait_info.ontology'))
 .select('gold_standard_info.gene_id', 'trait_id')
 .distinct()
)

# 619 pairs in the OT's GS with a high, medium confidence

In [38]:
(ot_gs.show())

+---------------+-----------+
|        gene_id|   trait_id|
+---------------+-----------+
|ENSG00000198721|HMDB0013205|
|ENSG00000107611|HMDB0002174|
|ENSG00000258947|EFO_0000707|
|ENSG00000015520|EFO_0004611|
|ENSG00000093217|HMDB0000115|
|ENSG00000134333|HMDB0000407|
|ENSG00000163931|HMDB0001429|
|ENSG00000167910|HMDB0000067|
|ENSG00000137834|EFO_0009387|
|ENSG00000100075|HMDB0000094|
|ENSG00000090857|HMDB0000243|
|ENSG00000137869|EFO_1001512|
|ENSG00000087085|EFO_0000289|
|ENSG00000136997|EFO_0001663|
|ENSG00000215182|EFO_0009824|
|ENSG00000159216|EFO_0007985|
|ENSG00000138207|HMDB0000305|
|ENSG00000151790|HMDB0000929|
|ENSG00000133019|EFO_0000341|
|ENSG00000061918|EFO_0000319|
+---------------+-----------+
only showing top 20 rows



In [187]:
## 1.2. Expand ontology

disease_idx = (
    spark.read.parquet('/Users/irene/Documents/dev/pyspark/21.09.5/diseases')
    .select(F.array(F.col('id')).alias('id'), 'ancestors')
    .withColumn('ids', F.array_union(F.col('id'), F.col('ancestors')))
    .withColumn('id', F.explode('id'))
    .select('id', 'ids')
)

disease_idx.first()

Row(id='GO_0044238', ids=['GO_0044238', 'GO_0008152', 'GO_0008150'])

In [188]:
ot_gs_expanded = (
    ot_gs.join(disease_idx, disease_idx['id'] == ot_gs['trait_id'], how='left')
    .withColumn('disease_id', F.coalesce('ids', F.array(F.col('trait_id'))))
    .withColumn('disease_id', F.explode('disease_id'))
    .select('gene_id', 'disease_id')
    .distinct()
    .persist()
)

ot_gs_expanded.show()

+---------------+---------------+
|        gene_id|     disease_id|
+---------------+---------------+
|ENSG00000198721|    HMDB0013205|
|ENSG00000107611|    HMDB0002174|
|ENSG00000258947|    EFO_0000707|
|ENSG00000115380|    EFO_0000618|
|ENSG00000145777|    EFO_0009433|
|ENSG00000154122|  MONDO_0001933|
|ENSG00000134243|    EFO_0004732|
|ENSG00000100344|    EFO_0010284|
|ENSG00000157978|    EFO_0001444|
|ENSG00000122691|     HP_0011025|
|ENSG00000088832|   OTAR_0000020|
|ENSG00000100985|    EFO_0006858|
|ENSG00000257923|   OTAR_0000010|
|ENSG00000015520|    EFO_0004611|
|ENSG00000093217|    HMDB0000115|
|ENSG00000113302|Orphanet_158300|
|ENSG00000258947|  MONDO_0000653|
|ENSG00000134827|    EFO_0001069|
|ENSG00000113580|  MONDO_0007263|
|ENSG00000171094|    EFO_0000311|
+---------------+---------------+
only showing top 20 rows



In [171]:
## 2. Eric's GS

file = 'data/manual_gene_to_trait_rules.xlsx'

eric_gs = spark.createDataFrame(pd.read_excel(file, sheet_name='Sheet1').astype(str))
eric_gs_2.first()

Row(type='metQTL', assertion_source='Fauman metabolites', published='yes', gene='A4GALT', trait='HMDB0011594|HMDB0011595', confidence='1000', why='A4GALT encodes a lactosylceramide 4-alpha-galactosyltransferase which adds a galactose moeity to a lactosylceramide molecule resulting in  globotriaosylceramide\xa0(Gb3). The observed metabolite is a likely substrate.', source='10748143')

In [172]:
eric_gs = (
    
    eric_gs.filter(F.col('gene').isNotNull())
    
    .filter(F.col('confidence') == '1000')
    
    # some metabolites are mapped to a less specific term.
    # That is indicated with a "~" before the ID (~HMDB0008047).
    # I take those mappings are valid and remove the '~'
    .withColumn('trait_id', F.translate('trait', '~+', ''))
    
    # Multiple gene/disease traits are divided by '|'
    .withColumn('trait_id', F.explode(F.split(F.col('trait_id'), '\|')))
    .withColumn('gene', F.explode(F.split(F.col('gene'), '\|')))
    
    .select('trait_id', 'gene').distinct()
)

eric_gs.show()

+-----------+-------+
|   trait_id|   gene|
+-----------+-------+
|HMDB0001438|    ABO|
|HMDB0001291|  ACADM|
|HMDB0011738|  ACADM|
|     CXCL11|  ACKR1|
|HMDB0000625|   ADH6|
|HMDB0007973|  APOA4|
|HMDB0000043|   CPS1|
|HMDB0001032|CYP2C18|
|HMDB0000554| CYP3A5|
|HMDB0008039|  FADS2|
|HMDB0010382|  FADS2|
|HMDB0007874|  FADS2|
|EFO_0005036|  GATA1|
|EFO_0004324|   GDF5|
|HMDB0000407|   HAO2|
|HMDB0000067|  PPARG|
|EFO_0004226|   PRNP|
|HMDB0000158|  PRODH|
|EFO_0004208|SLC24A5|
|       MSTN|WFIKKN2|
+-----------+-------+
only showing top 20 rows



In [141]:
## 2.1. Map gene to Ensembl

target_idx = (
    spark.read.parquet('/Users/irene/Documents/dev/pyspark/21.09.5/targets')
    .select(F.col('id').alias('gene_id'), F.col('approvedSymbol').alias('gene'))
)
target_idx.first()

Row(gene_id='ENSG00000002016', gene='RAD52')

In [173]:
# All genes except one (813/814) are mappable using the approved symbol 

eric_gs = eric_gs.join(target_idx, on='gene', how='left')
eric_gs.show()

+-------+-----------+---------------+
|   gene|   trait_id|        gene_id|
+-------+-----------+---------------+
|    ABO|HMDB0001438|ENSG00000175164|
|  ACADM|HMDB0001291|ENSG00000117054|
|  ACADM|HMDB0011738|ENSG00000117054|
|  ACKR1|     CXCL11|ENSG00000213088|
|   ADH6|HMDB0000625|ENSG00000172955|
|  APOA4|HMDB0007973|ENSG00000110244|
|   CPS1|HMDB0000043|ENSG00000021826|
|CYP2C18|HMDB0001032|ENSG00000108242|
| CYP3A5|HMDB0000554|ENSG00000106258|
|  FADS2|HMDB0008039|ENSG00000134824|
|  FADS2|HMDB0010382|ENSG00000134824|
|  FADS2|HMDB0007874|ENSG00000134824|
|  GATA1|EFO_0005036|ENSG00000102145|
|   GDF5|EFO_0004324|ENSG00000125965|
|   HAO2|HMDB0000407|ENSG00000116882|
|  PPARG|HMDB0000067|ENSG00000132170|
|   PRNP|EFO_0004226|ENSG00000171867|
|  PRODH|HMDB0000158|ENSG00000100033|
|SLC24A5|EFO_0004208|ENSG00000188467|
|WFIKKN2|       MSTN|ENSG00000173714|
+-------+-----------+---------------+
only showing top 20 rows



In [174]:
## 2.2 Expand ontology

eric_gs_expanded = (
    eric_gs.join(diseases, diseases['id'] == eric_gs['trait_id'], how='left')
    .withColumn('disease_id', F.coalesce('ids', F.array(F.col('trait_id'))))
    .withColumn('disease_id', F.explode('disease_id'))
    .select('gene_id', 'disease_id')
    .distinct()
    .persist()
)

eric_gs_expanded.show()



+---------------+-------------+
|        gene_id|   disease_id|
+---------------+-------------+
|ENSG00000196344|  EFO_0000618|
|ENSG00000157978|  EFO_0001444|
|ENSG00000177150|  EFO_0004260|
|ENSG00000242515|  HMDB0000054|
|ENSG00000132855|  HMDB0009815|
|ENSG00000110090|  HMDB0001043|
|ENSG00000162552|MONDO_0000632|
|ENSG00000169083|  EFO_0004191|
|ENSG00000158865|  HMDB0006088|
|ENSG00000172955|  EFO_0001444|
|ENSG00000134824|  HMDB0010371|
|ENSG00000100344|  EFO_0010284|
|ENSG00000042832|MONDO_0000569|
|ENSG00000134824|  HMDB0010392|
|ENSG00000130203|  EFO_0007796|
|ENSG00000126821|  HMDB0240616|
|ENSG00000235169|  EFO_0001444|
|ENSG00000119866|MONDO_0002280|
|ENSG00000164532|  EFO_0005278|
|ENSG00000269858|         HBA2|
+---------------+-------------+
only showing top 20 rows



## Questions

### 1. What is the overlap between GSs?

In [144]:

overlap = eric_gs.join(ot_gs, on=['trait_id', 'gene_id'], how='inner')
overlap_w_expansion = eric_gs_expanded.join(ot_gs_expanded, on=['disease_id', 'gene_id'], how='inner')

print('Assocs without expansion: ', overlap.count(), f'({overlap.count()/eric_gs.count():.3f})')
print('Assocs with expansion: ', overlap_w_expansion.count(), f'({overlap_w_expansion.count()/eric_gs_expanded.count():.3f})')

Assocs without expansion:  288 (0.079)
Assocs with expansion:  1041 (0.146)


### 2. Assess the value of this set as an evidence by seeing the overlap wth Clinvar.

In [175]:

clinvar = (
    spark.read.parquet('/Users/irene/Documents/dev/pyspark/21.09.5/associationByDatasourceIndirect')
    .filter(F.col('datasourceId') == 'eva')
    .select(
        F.col('targetId').alias('gene_id'),
        F.col('diseaseId').alias('disease_id'))
) 

clinvar.first()

Row(gene_id='ENSG00000085999', disease_id='EFO_0000574')

In [179]:
# Overlap with the expanded dataset as the assocs are indirect 

clinvar_overlap = eric_gs_expanded.join(clinvar, on=['disease_id', 'gene_id'], how='inner').distinct()

print('Erics assocs already in Clinvar: ', clinvar_overlap.count(), f'({clinvar_overlap.count()/eric_gs_expanded.count():.3f})')

Erics assocs already in Clinvar:  1107 (0.136)


### 3. Assess the value of this set as an evidence by seeing the overlap with OT Platform.

In [182]:
# Clinvar only covers a small amount of the GS. Let's make the comparison with the all the sources

assocs = (
    spark.read.parquet('/Users/irene/Documents/dev/pyspark/21.09.5/associationByDatasourceIndirect')
    .select(
        F.col('targetId').alias('gene_id'),
        F.col('diseaseId').alias('disease_id'))
) 

ot_overlap = eric_gs_expanded.join(assocs, on=['disease_id', 'gene_id'], how='inner').distinct()

print('Erics assocs already in OT: ', ot_overlap.count(), f'({ot_overlap.count()/eric_gs_expanded.count():.3f})')

Erics assocs already in OT:  5333 (0.654)


In [194]:
# Are Eric's novelties compatible with the Platform?

eric_novel = eric_gs_expanded.join(assocs, on=['disease_id', 'gene_id'], how='left_anti').distinct()

eric_resolvable_assocs = eric_novel.join(disease_idx, eric_novel['disease_id'] == disease_idx['id'], how='inner').drop('id', 'ids').distinct()

print('Potential Erics novel associations: ', eric_novel.count())
print(f'... of which {eric_resolvable_assocs.count()} ({eric_resolvable_assocs.count()/eric_novel.count():.3f}) can be resolved to OTs entities')

Potential Erics novel associations:  2818
... of which 421 (0.149) can be resolved to OTs entities


In [203]:
# Where are the non resolvable diseases coming from?
eric_nonresolvable_traits = eric_novel.join(disease_idx, eric_novel['disease_id'] == disease_idx['id'], how='left_anti').select('disease_id').distinct()
print(f"{eric_nonresolvable_traits.filter(F.col('disease_id').startswith('HMDB')).count()} / {eric_nonresolvable_traits.count()} refer to metabolites (HMDB terms).")

711 / 770 refer to metabolites (HMDB terms).


In [246]:
eric_nonresolvable_traits.first()

Row(disease_id='HMDB0011260')

### 4. Benchmark the locus2gene score

In [115]:
## 2. Benchmark the locus2gene score with the dataset
Likelihood of a gene is causal is to a certain trait.
Is the assignment Eric is making 

24118

In [216]:
# Read the table with the inferred **causal gene** of a **variant** for a certain **study**

l2g_table = spark.read.parquet('data/l2g.full.200127.parquet')
l2g_table.first()

Row(study_id='NEALE2_20015_raw', chrom='11', pos=66882056, ref='G', alt='C', gene_id='ENSG00000248643', training_clf='xgboost', training_gs='high_medium', training_fold='fold2=20|5|11|18', y_proba_dist_foot=0.016477391123771667, y_proba_dist_tss=0.03394272178411484, y_proba_full_model=0.025197738781571388, y_proba_logi_distance=0.02190273441374302, y_proba_logi_interaction=0.04681830853223801, y_proba_logi_molecularQTL=0.05755378678441048, y_proba_logi_pathogenicity=0.1470467895269394, y_proba_logo_distance=0.022010287269949913, y_proba_logo_interaction=0.017333192750811577, y_proba_logo_molecularQTL=0.021643899381160736, y_proba_logo_pathogenicity=0.021079089492559433)

In [250]:
l2g_table_processed = (
    
    l2g_table.withColumn('variant_id', F.concat('chrom', F.lit('_'), 'pos', F.lit('_'), 'ref', F.lit('_'), 'alt'))
    .withColumn('prediction', F.struct(F.col('gene_id').alias('gene_id'), F.col('y_proba_full_model').alias('proba')))
    
    .groupBy('variant_id', 'study_id')
    .agg(F.collect_set('prediction').alias('prediction'), F.max('y_proba_full_model').alias('max_proba'))
    
    # take gene_id with the greatest proba
    .withColumn('prediction', F.explode('prediction'))
    .filter(F.col('prediction.proba') == F.col('max_proba'))
    .withColumn('gene_id', F.col('prediction.gene_id'))
    
    # filter associations with a l2g score > .5
    .filter(F.col('max_proba') >= 0.5)
    
    .select('study_id', 'variant_id', 'gene_id')
    .distinct()
)

l2g_table_processed.first()

Row(study_id='GCST005306', variant_id='10_103565017_C_T', gene_id='ENSG00000107954')

In [251]:
# Extract associated trait for the study_id

study_idx = spark.read.json('/Users/irene/study-index').select('study_id', 'trait_efos')

l2g_table_processed_w_trait = (
    
    l2g_table_processed
    .join(study_idx, on='study_id', how='left')
    .withColumn('trait_id', F.explode('trait_efos'))
    
    .drop('trait_efos', 'study_id')
)

l2g_table_processed_w_trait.first()

Row(variant_id='10_103565017_C_T', gene_id='ENSG00000107954', trait_id='EFO_0000275')

In [252]:
# Ontology expansion to make it comparable with Eric's GS

l2g_table_processed_w_trait_expanded = (
    
    l2g_table_processed_w_trait.join(disease_idx, disease_idx['id'] == l2g_table_processed_w_trait['trait_id'], how='left')
    .withColumn('disease_id', F.coalesce('ids', F.array(F.col('trait_id'))))
    .withColumn('disease_id', F.explode('disease_id'))
    .select(F.col('gene_id').alias('predicted_gene_id'), 'disease_id')
    .distinct()
    .persist()
    
)

l2g_table_processed_w_trait_expanded.count()

165088

In [253]:
# How well does l2g predictions perform against the GS?

benchmark = (
    eric_gs_expanded.join(l2g_table_processed_w_trait_expanded, on='disease_id', how='left')
    .withColumn(
        'exactMatch',
        F.when(F.col('gene_id') == F.col('predicted_gene_id'), True).otherwise(False)
    )
    .distinct()
)

benchmark.first()

Row(disease_id='EFO_0000618', gene_id='ENSG00000196344', predicted_gene_id='ENSG00000143786', exactMatch=False)

In [254]:
benchmark.filter(F.col('exactMatch')).select('disease_id', 'gene_id').distinct().count()

print(f"{benchmark.filter(F.col('exactMatch')).select('disease_id', 'gene_id').distinct().count()} out of {benchmark.select('disease_id', 'gene_id').distinct().count()} Eric's gene/trait pairs overlap with those inferred from L2G.")

3311 out of 8151 Eric's gene/trait pairs overlap with those inferred from L2G.
